In [1]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import gc
import numpy as np
import config
import tensorflow as tf
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import merlin.models.tf as mm
from merlin.models.tf.core.aggregation import SequenceAggregator


dmodel = 64
BATCH_SIZE = 1024
LEARNING_RATE = 0.005
TOP = 10

2023-09-07 16:19:43.755842: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-07 16:19:43.779000: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[INFO]: sparse_operation_kit is imported


2023-09-07 16:19:46.207493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-07 16:19:46.207735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-07 16:19:46.207780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-09-07 16:19:46.419221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-07 16:19:46.419311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-07 16:19:46.419352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [22]:
train = Dataset(os.path.join(config.data_final,'transformer4rec', 'train.parquet/*.parquet'),engine="parquet",  part_size="1GB")
valid = Dataset(os.path.join(config.data_final,'transformer4rec', 'valid.parquet/*.parquet'),engine="parquet", part_size="1GB")

In [3]:
target_schema = train.schema.select_by_tag(Tags.ITEM_ID).column_names[0]


train.schema = train.schema.select_by_tag(Tags.SEQUENCE).select_by_tag(Tags.CATEGORICAL)
valid.schema = train.schema


In [4]:
target = train.schema.select_by_tag(Tags.ITEM_ID).column_names[0]
target

'article_id_list'

In [5]:
input_block = mm.InputBlockV2(
    train.schema,    
    embeddings=mm.Embeddings(
        train.schema, 
        sequence_combiner=None,
        dim=dmodel
        ),
)

In [6]:
batch = mm.sample_batch(train, batch_size=BATCH_SIZE, include_targets=False)
print(input_block(batch).shape)

(28, None, 1280)


In [7]:
input_block(batch).shape

TensorShape([28, None, 1280])

In [8]:
mlp_block = mm.MLPBlock(
                [128,dmodel],
                activation='relu',
                no_activation_last_layer=True,
            )

In [9]:
mlp_block(input_block(batch)).shape

2023-09-07 16:19:47.840828: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


TensorShape([28, None, 64])

In [10]:
xlnet_block = mm.XLNetBlock(d_model=dmodel, n_head=4, n_layer=2)

In [11]:
dense_block = mm.SequentialBlock(
    input_block,
    mlp_block,
    xlnet_block
)

In [12]:
item_id_name = train.schema.select_by_tag(Tags.ITEM_ID).first.properties['domain']['name']
print(item_id_name)

article_id


In [13]:
prediction_task = mm.CategoricalOutput(
    to_call=input_block["categorical"][item_id_name],
)

In [14]:
mlp_block2 = mm.MLPBlock(
                [128,dmodel],
                activation='relu',
                no_activation_last_layer=True,
            )

In [15]:
model_transformer = mm.Model(dense_block, mlp_block2, prediction_task)

In [16]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
)

In [18]:

model_transformer.compile(run_eagerly=False, optimizer=optimizer, loss="categorical_crossentropy",
              metrics=mm.TopKMetricsAggregator.default_metrics(top_ks=[10])
             )
model_transformer.fit(train, batch_size=512, epochs=30, pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block))

Epoch 1/30


2023-09-07 16:21:23.356687: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 4s 4s/step - loss: 10.6722 - recall_at_10: 0.0282 - mrr_at_10: 0.0115 - ndcg_at_10: 0.0154 - map_at_10: 0.0115 - precision_at_10: 0.0028 - regularization_loss: 0.0000e+00 - loss_batch: 10.6722
Epoch 2/30
1/1 [==============================] - 0s 176ms/step - loss: 9.9656 - recall_at_10: 0.0865 - mrr_at_10: 0.0428 - ndcg_at_10: 0.0530 - map_at_10: 0.0428 - precision_at_10: 0.0086 - regularization_loss: 0.0000e+00 - loss_batch: 9.9656
Epoch 3/30
1/1 [==============================] - 0s 180ms/step - loss: 8.7319 - recall_at_10: 0.0846 - mrr_at_10: 0.0386 - ndcg_at_10: 0.0494 - map_at_10: 0.0386 - precision_at_10: 0.0085 - regularization_loss: 0.0000e+00 - loss_batch: 8.7319
Epoch 4/30
1/1 [==============================] - 0s 178ms/step - loss: 7.0438 - recall_at_10: 0.0620 - mrr_at_10: 0.0364 - ndcg_at_10: 0.0424 - map_at_10: 0.0364 - precision_at_10: 0.0062 - regularization_loss: 0.0000e+00 - loss_batch: 7.0438
Epoch 5/30
1/1 [====================

In [ ]:
model_transformer.evaluate(
    valid,
    batch_size=1024,
    pre=mm.SequencePredictNext(schema=train.schema, target=target, transformer=xlnet_block),
    return_dict=True
)

2023-09-07 16:17:18.018682: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 4s 4s/step - loss: 1.6604 - recall_at_10: 0.9605 - mrr_at_10: 0.8917 - ndcg_at_10: 0.9096 - map_at_10: 0.8917 - precision_at_10: 0.0961 - regularization_loss: 0.0000e+00 - loss_batch: 1.6604


{'loss': 1.6604058742523193,
 'recall_at_10': 0.9605262875556946,
 'mrr_at_10': 0.8917292952537537,
 'ndcg_at_10': 0.9095906019210815,
 'map_at_10': 0.8917292952537537,
 'precision_at_10': 0.0960526391863823,
 'regularization_loss': 0.0,
 'loss_batch': 1.6604058742523193}

In [23]:
train.to_ddf().compute()

KeyError: 'Column price_min does not exist in schema'